This is a record of the initial Data Processing when the project was first began. As I went through the project I decided that ranking.csv is all based on cumulative averages and is not really needed as these and other rolling averages can be calculated from the games.csv data. I also decided to hold off initially using the individual player stats in games_details.csv. I may add select features from this data later.

The functions developed in this notebook were added to data_processing.py in the src folder for reusability

# Data Processing

### Summary

Games.csv and ranking.csv will be merged after initial data processing. Since all the features are *post* game data (final score, winning percentage after the game, etc...) they cannot be used as predictors for the current game. All the features will be used as predictors for the "next game", so the data will need to adjusted so that the TARGET (HOME_TEAM_WINS) is in the same row as the predictors. 

The easiest approach seems to be add a field called TARGET that denotes whether the home team won its *next* game or not.

Game_details.csv will intially be held in reserve for feature engineering. With a roster of 24 players per game and 21 features per player, the initial plan is NOT to add all these 500 features indiscriminately but to instead try to find useful features and incorporate these.

Scaling and power-transforms will not be used at this time since the plan is to use GBTs (gradient boosted trees) such as XGBoost where this is not needed. These transforms may be needed later for PCA and other techniques.


duplicates

 - both games.csv and ranking.csv contain several duplicated rows from Dec 2020 (covid season) that the pandas function *df.duplicated()* failed to detect in EDA. These will be filtered out using subsets instead of the entire dataframe.

 games.csv
 
 - delete preseason games (this will also take care of the null games from early 2003)
 - keep only games where GAME_STATUS_TEXT = 'Final' (for better utility in the future)
 - remove duplicated records 
 - flag postseason games 
 - drop 'GAME_STATUS_TEXT', 'TEAM_ID_home', 'TEAM_ID_away'

ranking.csv
 
 - drop preseason rankings (SEASON_ID begins with 1)
 - split HOME_RECORD into HOME_W, HOME_L, and HOME_W_PCT
 - split ROAD_RECORD into ROAD_W, ROAD_L, and ROAD_W_PCT
 - numericaly encode CONFERENCE (East or West)
 - remove duplicated records
 - drop 'SEASON_ID', 'LEAGUE_ID', 'RETURNTOPLAY', 'TEAM', 'HOME_RECORD', 'ROAD_RECORD'

 game_details.csv
 
 - fix mixed formats in MIN and convert to float
 - fix negatives in MIN
 - if MIN is null, edit START_POSITION to 'NP' (not played)
 - any START_POSITION remaining null, convert to NS (not start, but still played)
 - drop TEAM_ABBREVIATION, TEAM_CITY, PLAYER_NAME, NICKNAME, COMMENT
 
 Join games with ranking
 
  - LINK: games.GAME_DATE_EST, games.HOME_TEAM_ID, -> ranking.STANDINGSDATE, ranking.TEAM_ID 
  - ADD: CONFERENCE, G, W, L, W_PCT, HOME_W, HOME_L, HOME_W_PCT, ROAD_W, ROAD_L, ROAD_W_PCT
  - repeat with AWAY_TEAM_ID instead of HOME_TEAM_ID
  
 Add TARGET
 
  - Sort games by HOME_TEAM_ID and GAME_ID
  - for each SEASON and HOME_TEAM_ID, shift HOME_TEAM_WINS down to TARGET for previous game
  - remove games with null TARGETs (last game played each season by each team will have no null TARGET)
  

 

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 500)

# For Visualization
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path  #for Windows/Linux compatibility
DATAPATH = Path(r'data')


## games.csv

In [2]:
games = pd.read_csv(DATAPATH / "games.csv")
games.head()

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2022-03-12,22101005,Final,1610612748,1610612750,2021,1.610613e+09,104.0,0.398,0.760,0.333,23.0,53.0,1.610613e+09,113.0,0.422,0.875,0.357,21.0,46.0,0
1,2022-03-12,22101006,Final,1610612741,1610612739,2021,1.610613e+09,101.0,0.443,0.933,0.429,20.0,46.0,1.610613e+09,91.0,0.419,0.824,0.208,19.0,40.0,1
2,2022-03-12,22101007,Final,1610612759,1610612754,2021,1.610613e+09,108.0,0.412,0.813,0.324,28.0,52.0,1.610613e+09,119.0,0.489,1.000,0.389,23.0,47.0,0
3,2022-03-12,22101008,Final,1610612744,1610612749,2021,1.610613e+09,122.0,0.484,0.933,0.400,33.0,55.0,1.610613e+09,109.0,0.413,0.696,0.386,27.0,39.0,1
4,2022-03-12,22101009,Final,1610612743,1610612761,2021,1.610613e+09,115.0,0.551,0.750,0.407,32.0,39.0,1.610613e+09,127.0,0.471,0.760,0.387,28.0,50.0,0


**Clean Data**

In [3]:
#remove preseason games (GAME_ID begins with a 1)
games = games[games['GAME_ID'] > 20000000]

#flag postseason games (GAME_ID begins with >2)
games['PLAYOFF'] = (games['GAME_ID'] >= 30000000).astype('int8')

#remove duplicates (each GAME_ID should be unique)
games = games[~games.duplicated(subset=['GAME_ID'])]

#drop unnecessary fields
drop_fields = ['GAME_STATUS_TEXT', 'TEAM_ID_home', 'TEAM_ID_away']
games = games.drop(drop_fields,axis=1)

games
    

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,PLAYOFF
0,2022-03-12,22101005,1610612748,1610612750,2021,104.0,0.398,0.760,0.333,23.0,53.0,113.0,0.422,0.875,0.357,21.0,46.0,0,0
1,2022-03-12,22101006,1610612741,1610612739,2021,101.0,0.443,0.933,0.429,20.0,46.0,91.0,0.419,0.824,0.208,19.0,40.0,1,0
2,2022-03-12,22101007,1610612759,1610612754,2021,108.0,0.412,0.813,0.324,28.0,52.0,119.0,0.489,1.000,0.389,23.0,47.0,0,0
3,2022-03-12,22101008,1610612744,1610612749,2021,122.0,0.484,0.933,0.400,33.0,55.0,109.0,0.413,0.696,0.386,27.0,39.0,1,0
4,2022-03-12,22101009,1610612743,1610612761,2021,115.0,0.551,0.750,0.407,32.0,39.0,127.0,0.471,0.760,0.387,28.0,50.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26762,2022-10-19 00:00:00,22200004,1610612764,1610612754,2022,114.0,45.700,79.200,35.500,21.0,53.0,107.0,40.200,66.700,35.700,21.0,42.0,1,0
26763,2022-10-19 00:00:00,22200014,1610612757,1610612758,2022,115.0,44.300,78.800,39.300,20.0,44.0,108.0,45.900,68.400,38.600,27.0,41.0,1,0
26764,2022-10-19 00:00:00,22200003,1610612753,1610612765,2022,109.0,48.800,73.700,36.700,21.0,48.0,113.0,42.600,79.200,36.800,31.0,41.0,0,0
26765,2022-10-18 00:00:00,22200002,1610612747,1610612744,2022,109.0,42.600,76.000,25.000,23.0,48.0,123.0,45.500,73.900,35.600,31.0,48.0,0,0


## ranking.csv

In [4]:
ranking = pd.read_csv(DATAPATH / "ranking.csv")
ranking.head()

,TEAM_ID,LEAGUE_ID,SEASON_ID,STANDINGSDATE,CONFERENCE,TEAM,G,W,L,W_PCT,HOME_RECORD,ROAD_RECORD,RETURNTOPLAY
0,1610612756,0,22021,2022-03-12,West,Phoenix,67,53,14,0.791,28-8,25-6,NaN
1,1610612744,0,22021,2022-03-12,West,Golden State,68,46,22,0.676,28-7,18-15,NaN
2,1610612763,0,22021,2022-03-12,West,Memphis,68,46,22,0.676,24-10,22-12,NaN
3,1610612762,0,22021,2022-03-12,West,Utah,67,42,25,0.627,24-10,18-15,NaN
4,1610612742,0,22021,2022-03-12,West,Dallas,67,41,26,0.612,23-12,18-14,NaN


**Clean Data**

In [5]:
#remove preseason rankings (SEASON_ID begins with 1)
ranking = ranking[ranking['SEASON_ID'] > 20000]

#convert home record and road record to numeric
ranking['HOME_W'] = ranking['HOME_RECORD'].apply(lambda x: x.split('-')[0]).astype('int')
ranking['HOME_L'] = ranking['HOME_RECORD'].apply(lambda x: x.split('-')[1]).astype('int')
ranking['HOME_W_PCT'] = ranking['HOME_W'] / ( ranking['HOME_W'] + ranking['HOME_L'] )

ranking['ROAD_W'] = ranking['ROAD_RECORD'].apply(lambda x: x.split('-')[0]).astype('int')
ranking['ROAD_L'] = ranking['ROAD_RECORD'].apply(lambda x: x.split('-')[1]).astype('int')
ranking['ROAD_W_PCT'] = ranking['ROAD_W'] / ( ranking['ROAD_W'] + ranking['ROAD_L'] )

#encode CONFERENCE as an integer (just using pandas - not importing sklearn for just one feature)
ranking['CONFERENCE'] = ranking['CONFERENCE'].apply(lambda x: 0 if x=='East' else 1 ).astype('int') 

#remove duplicates (there should only be one TEAM_ID per STANDINGSDATE)
ranking = ranking[~ranking.duplicated(subset=['TEAM_ID','STANDINGSDATE'])]

#drop unnecessary fields
drop_fields = ['SEASON_ID', 'LEAGUE_ID', 'RETURNTOPLAY', 'TEAM', 'HOME_RECORD', 'ROAD_RECORD']
ranking = ranking.drop(drop_fields,axis=1)

ranking


,TEAM_ID,STANDINGSDATE,CONFERENCE,G,W,L,W_PCT,HOME_W,HOME_L,HOME_W_PCT,ROAD_W,ROAD_L,ROAD_W_PCT
0,1610612756,2022-03-12,1,67,53,14,0.791,28,8,0.777778,25,6,0.806452
1,1610612744,2022-03-12,1,68,46,22,0.676,28,7,0.800000,18,15,0.545455
2,1610612763,2022-03-12,1,68,46,22,0.676,24,10,0.705882,22,12,0.647059
3,1610612762,2022-03-12,1,67,42,25,0.627,24,10,0.705882,18,15,0.545455
4,1610612742,2022-03-12,1,67,41,26,0.612,23,12,0.657143,18,14,0.562500
...,...,...,...,...,...,...,...,...,...,...,...,...,...
201787,1610612765,2014-09-01,0,82,29,53,0.354,17,24,0.414634,12,29,0.292683
201788,1610612738,2014-09-01,0,82,25,57,0.305,16,25,0.390244,9,32,0.219512
201789,1610612753,2014-09-01,0,82,23,59,0.280,19,22,0.463415,4,37,0.097561
201790,1610612755,2014-09-01,0,82,19,63,0.232,10,31,0.243902,9,32,0.219512


## game_details.csv

In [6]:
details = pd.read_csv(DATAPATH / "games_details.csv")
details.head()

C:\Users\Chris\AppData\Local\Temp\ipykernel_1704\2262652934.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  details = pd.read_csv(DATAPATH / "games_details.csv")


,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,22101005,1610612750,MIN,Minnesota,1630162,Anthony Edwards,Anthony,F,NaN,36:22,4.0,10.0,0.400,3.0,8.0,0.375,4.0,4.0,1.00,0.0,8.0,8.0,5.0,3.0,1.0,1.0,1.0,15.0,5.0
1,22101005,1610612750,MIN,Minnesota,1630183,Jaden McDaniels,Jaden,F,NaN,23:54,6.0,8.0,0.750,1.0,3.0,0.333,1.0,1.0,1.00,2.0,4.0,6.0,0.0,0.0,2.0,2.0,6.0,14.0,10.0
2,22101005,1610612750,MIN,Minnesota,1626157,Karl-Anthony Towns,Karl-Anthony,C,NaN,25:17,4.0,9.0,0.444,1.0,3.0,0.333,6.0,8.0,0.75,1.0,9.0,10.0,0.0,0.0,0.0,3.0,4.0,15.0,14.0
3,22101005,1610612750,MIN,Minnesota,1627736,Malik Beasley,Malik,G,NaN,30:52,4.0,9.0,0.444,4.0,9.0,0.444,0.0,0.0,0.00,0.0,3.0,3.0,1.0,1.0,0.0,1.0,4.0,12.0,20.0
4,22101005,1610612750,MIN,Minnesota,1626156,D'Angelo Russell,D'Angelo,G,NaN,33:46,3.0,13.0,0.231,1.0,6.0,0.167,7.0,7.0,1.00,0.0,6.0,6.0,9.0,1.0,0.0,5.0,0.0,14.0,17.0


**Clean Data**

In [7]:
# convert MIN:SEC to float
df = details.loc[details['MIN'].str.contains(':',na=False)]
df['MIN_whole'] = df['MIN'].apply(lambda x: x.split(':')[0]).astype("int8")
df['MIN_seconds'] = df['MIN'].apply(lambda x: x.split(':')[1]).astype("int8")
df['MIN'] = df['MIN_whole'] + (df['MIN_seconds'] / 60)

details['MIN'].loc[details['MIN'].str.contains(':',na=False)] = df['MIN']
details['MIN'] = details['MIN'].astype("float16")

# convert negatives to positive
details['MIN'].loc[details['MIN'] < 0] = -(details['MIN'])

#update START_POSITION if did not play (MIN = NaN)
details['START_POSITION'].loc[details['MIN'].isna()] = 'NP'

#update START_POSITION if null
details['START_POSITION'] = details['START_POSITION'].fillna('NS')

#drop unnecessary fields
drop_fields = ['COMMENT', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_NAME', 'NICKNAME'] 
details = details.drop(drop_fields,axis=1)

details

C:\Users\Chris\AppData\Local\Temp\ipykernel_1704\257855735.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MIN_whole'] = df['MIN'].apply(lambda x: x.split(':')[0]).astype("int8")
C:\Users\Chris\AppData\Local\Temp\ipykernel_1704\257855735.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MIN_seconds'] = df['MIN'].apply(lambda x: x.split(':')[1]).astype("int8")
C:\Users\Chris\AppData\Local\Temp\ipykernel_1704\257855735.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

,GAME_ID,TEAM_ID,PLAYER_ID,START_POSITION,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,22101005,1610612750,1630162,F,36.375000,4.0,10.0,0.400,3.0,8.0,0.375,4.0,4.0,1.000,0.0,8.0,8.0,5.0,3.0,1.0,1.0,1.0,15.0,5.0
1,22101005,1610612750,1630183,F,23.906250,6.0,8.0,0.750,1.0,3.0,0.333,1.0,1.0,1.000,2.0,4.0,6.0,0.0,0.0,2.0,2.0,6.0,14.0,10.0
2,22101005,1610612750,1626157,C,25.281250,4.0,9.0,0.444,1.0,3.0,0.333,6.0,8.0,0.750,1.0,9.0,10.0,0.0,0.0,0.0,3.0,4.0,15.0,14.0
3,22101005,1610612750,1627736,G,30.859375,4.0,9.0,0.444,4.0,9.0,0.444,0.0,0.0,0.000,0.0,3.0,3.0,1.0,1.0,0.0,1.0,4.0,12.0,20.0
4,22101005,1610612750,1626156,G,33.781250,3.0,13.0,0.231,1.0,6.0,0.167,7.0,7.0,1.000,0.0,6.0,6.0,9.0,1.0,0.0,5.0,0.0,14.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
645948,11200005,1610612743,202706,NS,19.000000,4.0,9.0,0.444,3.0,6.0,0.500,6.0,7.0,0.857,0.0,2.0,2.0,0.0,2.0,0.0,1.0,3.0,17.0,NaN
645949,11200005,1610612743,202702,NS,23.000000,7.0,11.0,0.636,0.0,0.0,0.000,4.0,4.0,1.000,1.0,0.0,1.0,1.0,1.0,0.0,3.0,3.0,18.0,NaN
645950,11200005,1610612743,201585,NS,15.000000,3.0,7.0,0.429,0.0,0.0,0.000,0.0,0.0,0.000,3.0,5.0,8.0,0.0,1.0,0.0,0.0,3.0,6.0,NaN
645951,11200005,1610612743,202389,NS,19.000000,1.0,1.0,1.000,0.0,0.0,0.000,0.0,2.0,0.000,1.0,2.0,3.0,1.0,0.0,0.0,4.0,2.0,2.0,NaN


## Add correct TARGET to dataframe

In [8]:
games['TARGET'] = games['HOME_TEAM_WINS']

games

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,PLAYOFF,TARGET
0,2022-03-12,22101005,1610612748,1610612750,2021,104.0,0.398,0.760,0.333,23.0,53.0,113.0,0.422,0.875,0.357,21.0,46.0,0,0,0
1,2022-03-12,22101006,1610612741,1610612739,2021,101.0,0.443,0.933,0.429,20.0,46.0,91.0,0.419,0.824,0.208,19.0,40.0,1,0,1
2,2022-03-12,22101007,1610612759,1610612754,2021,108.0,0.412,0.813,0.324,28.0,52.0,119.0,0.489,1.000,0.389,23.0,47.0,0,0,0
3,2022-03-12,22101008,1610612744,1610612749,2021,122.0,0.484,0.933,0.400,33.0,55.0,109.0,0.413,0.696,0.386,27.0,39.0,1,0,1
4,2022-03-12,22101009,1610612743,1610612761,2021,115.0,0.551,0.750,0.407,32.0,39.0,127.0,0.471,0.760,0.387,28.0,50.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26762,2022-10-19 00:00:00,22200004,1610612764,1610612754,2022,114.0,45.700,79.200,35.500,21.0,53.0,107.0,40.200,66.700,35.700,21.0,42.0,1,0,1
26763,2022-10-19 00:00:00,22200014,1610612757,1610612758,2022,115.0,44.300,78.800,39.300,20.0,44.0,108.0,45.900,68.400,38.600,27.0,41.0,1,0,1
26764,2022-10-19 00:00:00,22200003,1610612753,1610612765,2022,109.0,48.800,73.700,36.700,21.0,48.0,113.0,42.600,79.200,36.800,31.0,41.0,0,0,0
26765,2022-10-18 00:00:00,22200002,1610612747,1610612744,2022,109.0,42.600,76.000,25.000,23.0,48.0,123.0,45.500,73.900,35.600,31.0,48.0,0,0,0


**Save Train Data**

In [9]:
games.to_csv(DATAPATH / "transformed.csv",index=False)

**Sweetviz data visualization**

In [10]:
def run_sweetviz_report(df, TARGET):
    
    import sweetviz as sv
    from datetime import datetime
    
    report_label = datetime.today().strftime('%Y-%m-%d_%H_%M')
    
    my_report = sv.analyze(df,target_feat=TARGET)
    my_report.show_html(filepath='SWEETVIZ_' + report_label + '.html')
    
    return

df = pd.read_csv(DATAPATH / "transformed.csv")
run_sweetviz_report(df,'TARGET')

                                             |          | [  0%]   00:00 -> (? left)

Report SWEETVIZ_2023-01-31_10_15.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


**Correlations with TARGET**

In [11]:
df = games.drop(columns=['GAME_ID', 'TARGET'])

x = df.corrwith(games['TARGET'], method = 'spearman').sort_values(ascending=False)

print(x)

HOME_TEAM_WINS     1.000000
PTS_home           0.387400
FG_PCT_home        0.381894
AST_home           0.297406
FG3_PCT_home       0.265294
REB_home           0.246798
FT_PCT_home        0.060599
VISITOR_TEAM_ID    0.031611
PLAYOFF            0.025458
HOME_TEAM_ID      -0.026908
SEASON            -0.054614
FT_PCT_away       -0.122725
REB_away          -0.254399
AST_away          -0.292869
FG3_PCT_away      -0.297649
PTS_away          -0.422149
FG_PCT_away       -0.426479
dtype: float64
